This notebook documents part 2 of the complementarity of image and demographic information: the ability of latent space extracted from Autoencoders to predict mode choice and trip generation.

In [2]:
import sys
sys.path.append("models/")

%load_ext autoreload
%autoreload 2
from collections import OrderedDict
import os
import matplotlib.pyplot as plt
%matplotlib inline

import pandas as pd
import pickle as pkl
import numpy as np

import itertools
import glob

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from sklearn import linear_model
from sklearn.metrics import r2_score, mean_squared_error
import statsmodels.api as sm
from exp_version import get_hp_from_version_code


from dataloader import SurveyDataset, load_aggregate_travel_behavior, load_demo
from M1_util_train_test import load_model, test
import linear_reg
import mnl
from setup import out_dir, data_dir, image_dir, model_dir, proj_dir

plt.rcParams.update({"font.size":12})

In [1]:
from util_model import parse_args

zoomlevel = 'zoom15'
latent_dim = 64*2
demo_dim = 7
demo_cols = [0,1,2,3,4,8,9]
image_size = str(64)
im_norm = '2'
model_run_date = "2210gan-7"
model_type = 'dcgan'
loss_func = 'cosh'
model_class = 'vae'
base_lr = '0.0002'
weight_decay = '0'
sampling = 'clustered'

args = parse_args(s=['--zoomlevel', zoomlevel, '--latent_dim', str(latent_dim), 
                     '--image_size', image_size, \
                     '--im_norm', im_norm, '--model_run_date', model_run_date, '--model_type', model_type, \
                     '--loss_func', loss_func, '--model_class', model_class, \
                     '--base_lr', base_lr, '--weight_decay', weight_decay, \
                     '--demo_channels', str(demo_dim),
                     '--sampling', sampling])



# Load Model Embeddings

In [4]:
with open(proj_dir+"latent_space/cVAE_"+args.zoomlevel+"_"+str(args.latent_dim//2)+"_"+\
                      str(args.image_size)+"_"+str(int(args.im_norm))+"_"+str(args.model_run_date)+\
                   "_"+args.loss_func+".pkl", "rb") as f: 
    encoder_output = pkl.load(f)
    im = pkl.load(f)
    ct = pkl.load(f)
    test_encoder_output = pkl.load(f)
    test_im = pkl.load(f)
    test_ct = pkl.load(f)

In [5]:
# Aggregate Embeddings
unique_ct = list(set(ct))
unique_ct.sort()
ct = np.array(ct)
aggregate_embeddings = []
for i in unique_ct:
    aggregate_embeddings.append(np.mean(encoder_output[ct == i], axis=0))
aggregate_embeddings = np.array(aggregate_embeddings)

In [7]:
# Aggregate Test Embeddings
test_unique_ct = list(set(test_ct))
test_unique_ct.sort()
test_ct = np.array(test_ct)
test_aggregate_embeddings = []
for i in test_unique_ct:
    test_aggregate_embeddings.append(np.mean(test_encoder_output[test_ct == i], axis=0))
test_aggregate_embeddings = np.array(test_aggregate_embeddings)

In [10]:
aggregate_embeddings.shape

(1414, 64)

In [8]:
test_aggregate_embeddings.shape

(157, 64)

# Load Trip Behavior

In [20]:
file = "origin_trip_behavior.csv"
df_pivot = load_aggregate_travel_behavior(file, data_version='1571')

train_test_index = df_pivot['train_test'].astype(bool).to_numpy()
variable_names = ['active','auto','mas','pt', 'trpgen']

y = df_pivot[variable_names].to_numpy()
y_train = y[~train_test_index,:4]
y_test = y[train_test_index,:4]

In [21]:
x_train = aggregate_embeddings
x_test = test_aggregate_embeddings

In [22]:
auto_train = y[~train_test_index,1]
auto_test = y[train_test_index,1]

pt_train = y[~train_test_index,3]
pt_test = y[train_test_index,3]

active_train = y[~train_test_index,0]
active_test = y[train_test_index,0]

trpgen_train = y[~train_test_index,-1]
trpgen_test = y[train_test_index,-1]


# 1. Linear Regression

### 1.1 Auto Share

In [23]:
# Linear Regression without Regularization
lr = linear_model.LinearRegression()
lr.fit(x_train, auto_train)
# with open(out_dir+sampling+"_"+model_code+"_regression_"+variable_names[-1]+".csv", "a") as f:
#     f.write("%s,%s,%s,%.4f,%.4f,%.4f,%s,%s,%d,%d\n" % (model_run_date, model_type, variable_names[-1], -1, 
#         lr.score(x_train, auto_train), lr.score(x_test, auto_test), 'lr', zoomlevel,
#         np.sum(lr.coef_ != 0), len(lr.coef_)))
print("Train R2: %.4f \t Test R2: %.4f" % (lr.score(x_train, auto_train), lr.score(x_test, auto_test)))

Train R2: 0.2967 	 Test R2: 0.0067


In [28]:
# Lasso
for a in (1e-2)*np.array([0,0.1,0.2,0.4,0.6,0.8,1,2,3,4,5,6,7,8]):
    lasso = linear_model.Lasso(alpha=a)
    lasso.fit(x_train, auto_train)
    print("Parameter: %.2e Train R2: %.4f \t Test R: %.4f \t Nonzero coef: %d" % \
          (a, lasso.score(x_train, auto_train), lasso.score(x_test, auto_test), np.sum(lasso.coef_ != 0)))

#     with open(out_dir+"SAE_A_LR.csv", "a") as f:
#         f.write("%.2E,%.6f,%s,%.4f,%.4f,%s,%d,%d\n" % (weight,a,'auto',
#             lasso.score(x_train, auto_train), lasso.score(x_test, auto_test), 'lasso', 
#             np.sum(lasso.coef_ != 0), len(lasso.coef_)))


Parameter: 0.00e+00 Train R2: 0.2967 	 Test R: 0.0067 	 Nonzero coef: 64
Parameter: 1.00e-03 Train R2: 0.2781 	 Test R: 0.0832 	 Nonzero coef: 29
Parameter: 2.00e-03 Train R2: 0.2602 	 Test R: 0.0974 	 Nonzero coef: 14
Parameter: 4.00e-03 Train R2: 0.2447 	 Test R: 0.1055 	 Nonzero coef: 5
Parameter: 6.00e-03 Train R2: 0.2383 	 Test R: 0.1104 	 Nonzero coef: 4
Parameter: 8.00e-03 Train R2: 0.2336 	 Test R: 0.1219 	 Nonzero coef: 3
Parameter: 1.00e-02 Train R2: 0.2289 	 Test R: 0.1301 	 Nonzero coef: 3
Parameter: 2.00e-02 Train R2: 0.2025 	 Test R: 0.1571 	 Nonzero coef: 2
Parameter: 3.00e-02 Train R2: 0.1654 	 Test R: 0.1497 	 Nonzero coef: 1
Parameter: 4.00e-02 Train R2: 0.1518 	 Test R: 0.1394 	 Nonzero coef: 1
Parameter: 5.00e-02 Train R2: 0.1342 	 Test R: 0.1246 	 Nonzero coef: 1
Parameter: 6.00e-02 Train R2: 0.1127 	 Test R: 0.1055 	 Nonzero coef: 1
Parameter: 7.00e-02 Train R2: 0.0874 	 Test R: 0.0820 	 Nonzero coef: 1
Parameter: 8.00e-02 Train R2: 0.0581 	 Test R: 0.0540 	 Nonze

<ipython-input-28-ce4fb1c46106>:4: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
  lasso.fit(x_train, auto_train)
/home/jtl/anaconda3/envs/qingyi/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:647: UserWarning: Coordinate descent with no regularization may lead to unexpected results and is discouraged.
  model = cd_fast.enet_coordinate_descent(
/home/jtl/anaconda3/envs/qingyi/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.709e+01, tolerance: 7.704e-03 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in sklearn.linear_model.Ridge/RidgeCV instead.
  model = cd_fast.enet_coordinate_descent(


In [ ]:
# Ridge

for a in (5e+0)*np.array([0,0.1,1,2,3,4,5,6,7,8,10,20,50]):

    ridge = linear_model.Ridge(alpha=a)
    ridge.fit(x_train, auto_train)
#     with open(out_dir+sampling+"_"+model_code+"_regression_"+variable_names[-1]+".csv", "a") as f:
#         f.write("%s,%s,%s,%.5f,%.4f,%.4f,%s,%s,%d,%d\n" % (model_run_date, model_type, variable_names[-1], a, 
#             ridge.score(x_train, trpgen_train), ridge.score(x_test, trpgen_test), 'ridge', zoomlevel,
#             np.sum(ridge.coef_ != 0), len(ridge.coef_)))
    print("Parameter: %.2e Train R2: %.4f \t Test R: %.4f" % (a, ridge.score(x_train, auto_train), 
                                                              ridge.score(x_test, auto_test)))

### 1.2 PT

In [30]:
# Lasso
for a in (1e-2)*np.array([0,0.1,0.2,0.4,0.6,0.8,1,2,3,4,5,6,7,8,10,20,50]):
    lasso = linear_model.Lasso(alpha=a)
    lasso.fit(x_train, pt_train)
    print("Parameter: %.2e Train R2: %.4f \t Test R: %.4f \t Nonzero coef: %d" % (a, lasso.score(x_train, pt_train), 
                                                                                  lasso.score(x_test, pt_test), 
                                                                                  np.sum(lasso.coef_ != 0)))

#     with open(out_dir+"SAE_A_LR.csv", "a") as f:
#         f.write("%.2E,%.6f,%s,%.4f,%.4f,%s,%d,%d\n" % (weight,a,'pt',
#             lasso.score(x_train, pt_train), lasso.score(x_test, pt_test), 'lasso', 
#             np.sum(lasso.coef_ != 0), len(lasso.coef_)))

<ipython-input-30-7120b7093049>:4: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
  lasso.fit(x_train, pt_train)
/home/jtl/anaconda3/envs/qingyi/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:647: UserWarning: Coordinate descent with no regularization may lead to unexpected results and is discouraged.
  model = cd_fast.enet_coordinate_descent(


Parameter: 0.00e+00 Train R2: 0.2512 	 Test R: 0.0327 	 Nonzero coef: 64
Parameter: 1.00e-03 Train R2: 0.2016 	 Test R: 0.1087 	 Nonzero coef: 10
Parameter: 2.00e-03 Train R2: 0.1796 	 Test R: 0.1045 	 Nonzero coef: 4
Parameter: 4.00e-03 Train R2: 0.1727 	 Test R: 0.1089 	 Nonzero coef: 2
Parameter: 6.00e-03 Train R2: 0.1635 	 Test R: 0.1116 	 Nonzero coef: 2
Parameter: 8.00e-03 Train R2: 0.1506 	 Test R: 0.1058 	 Nonzero coef: 2
Parameter: 1.00e-02 Train R2: 0.1340 	 Test R: 0.0917 	 Nonzero coef: 2
Parameter: 2.00e-02 Train R2: 0.0997 	 Test R: 0.0749 	 Nonzero coef: 1
Parameter: 3.00e-02 Train R2: 0.0449 	 Test R: 0.0360 	 Nonzero coef: 1
Parameter: 4.00e-02 Train R2: -0.0000 	 Test R: -0.0000 	 Nonzero coef: 0
Parameter: 5.00e-02 Train R2: -0.0000 	 Test R: -0.0000 	 Nonzero coef: 0
Parameter: 6.00e-02 Train R2: -0.0000 	 Test R: -0.0000 	 Nonzero coef: 0
Parameter: 7.00e-02 Train R2: -0.0000 	 Test R: -0.0000 	 Nonzero coef: 0
Parameter: 8.00e-02 Train R2: -0.0000 	 Test R: -0.000

/home/jtl/anaconda3/envs/qingyi/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 5.138e+00, tolerance: 1.373e-03 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in sklearn.linear_model.Ridge/RidgeCV instead.
  model = cd_fast.enet_coordinate_descent(


In [ ]:
# Ridge

for a in (1e+1)*np.array([0,0.5,0.6,0.7,0.8,0.9,1,2,3,4,5,10,20,50]):
# for a in (5e0) * np.array([0, 4.5,4.6,4.7,4.8,4.9, 5, 5.1,5.2,5.3,5.4,5.5]):
    ridge = linear_model.Ridge(alpha=a)
    ridge.fit(x_train, pt_train)
#     with open(out_dir+sampling+"_"+model_code+"_regression_"+variable_names[-1]+".csv", "a") as f:
#         f.write("%s,%s,%s,%.5f,%.4f,%.4f,%s,%s,%d,%d\n" % (model_run_date, model_type, variable_names[-1], a, 
#             ridge.score(x_train, trpgen_train), ridge.score(x_test, trpgen_test), 'ridge', zoomlevel,
#             np.sum(ridge.coef_ != 0), len(ridge.coef_)))
    print("Parameter: %.2e Train R2: %.4f \t Test R: %.4f" % (a, ridge.score(x_train, pt_train), 
                                                              ridge.score(x_test, pt_test)))

### 1.3 Active

In [31]:
for a in (1e-2)*np.array([0,0.1,0.2,0.4,0.6,0.8,1,2,3,4,5,6,7,8,10,20,50]):
    lasso = linear_model.Lasso(alpha=a)
    lasso.fit(x_train, active_train)
    print("Parameter: %.2e Train R2: %.4f \t Test R: %.4f \t Nonzero coef: %d" % (a, lasso.score(x_train, active_train), 
                                                                                  lasso.score(x_test, active_test), 
                                                                                  np.sum(lasso.coef_ != 0)))

#     with open(out_dir+"SAE_A_LR.csv", "a") as f:
#         f.write("%.2E,%.6f,%s,%.4f,%.4f,%s,%d,%d\n" % (weight,a,'active',
#             lasso.score(x_train, active_train), lasso.score(x_test, active_test), 'lasso', 
#             np.sum(lasso.coef_ != 0), len(lasso.coef_)))

Parameter: 0.00e+00 Train R2: 0.2502 	 Test R: -0.0880 	 Nonzero coef: 64
Parameter: 1.00e-03 Train R2: 0.2250 	 Test R: 0.0072 	 Nonzero coef: 18
Parameter: 2.00e-03 Train R2: 0.2138 	 Test R: 0.0361 	 Nonzero coef: 7
Parameter: 4.00e-03 Train R2: 0.2022 	 Test R: 0.0563 	 Nonzero coef: 5
Parameter: 6.00e-03 Train R2: 0.1938 	 Test R: 0.0751 	 Nonzero coef: 3
Parameter: 8.00e-03 Train R2: 0.1863 	 Test R: 0.0922 	 Nonzero coef: 3
Parameter: 1.00e-02 Train R2: 0.1771 	 Test R: 0.1050 	 Nonzero coef: 2
Parameter: 2.00e-02 Train R2: 0.1289 	 Test R: 0.1307 	 Nonzero coef: 1
Parameter: 3.00e-02 Train R2: 0.1090 	 Test R: 0.1121 	 Nonzero coef: 1
Parameter: 4.00e-02 Train R2: 0.0813 	 Test R: 0.0844 	 Nonzero coef: 1
Parameter: 5.00e-02 Train R2: 0.0456 	 Test R: 0.0476 	 Nonzero coef: 1
Parameter: 6.00e-02 Train R2: 0.0019 	 Test R: 0.0018 	 Nonzero coef: 1
Parameter: 7.00e-02 Train R2: -0.0000 	 Test R: -0.0003 	 Nonzero coef: 0
Parameter: 8.00e-02 Train R2: -0.0000 	 Test R: -0.0003 	 N

<ipython-input-31-0aabbf89e3f7>:3: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
  lasso.fit(x_train, active_train)
/home/jtl/anaconda3/envs/qingyi/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:647: UserWarning: Coordinate descent with no regularization may lead to unexpected results and is discouraged.
  model = cd_fast.enet_coordinate_descent(
/home/jtl/anaconda3/envs/qingyi/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.421e+01, tolerance: 3.791e-03 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in sklearn.linear_model.Ridge/RidgeCV instead.
  model = cd_fast.enet_coordinate_descent

In [ ]:
# Ridge

# for a in (1e+1)*np.array([0,0.1,0.5,0.6,0.7,0.8,0.9,1,2,3,4,5,10,20,50]):
for a in (1e+1)*np.array([0,1, 1.5, 2, 1.5, 3]):
    
    ridge = linear_model.Ridge(alpha=a)
    ridge.fit(x_train, active_train)
#     with open(out_dir+sampling+"_"+model_code+"_regression_"+variable_names[-1]+".csv", "a") as f:
#         f.write("%s,%s,%s,%.5f,%.4f,%.4f,%s,%s,%d,%d\n" % (model_run_date, model_type, variable_names[-1], a, 
#             ridge.score(x_train, trpgen_train), ridge.score(x_test, trpgen_test), 'ridge', zoomlevel,
#             np.sum(ridge.coef_ != 0), len(ridge.coef_)))
    print("Parameter: %.2e Train R2: %.4f \t Test R: %.4f" % (a, ridge.score(x_train, active_train), 
                                                              ridge.score(x_test, active_test)))

### 1.4 Trip Generation

In [33]:
for a in (1e-3)*np.array([0,0.1,6,7,8,10,11,12,13,14,15,20,50]):
    lasso = linear_model.Lasso(alpha=a)
    lasso.fit(x_train, trpgen_train)
    print("Parameter: %.2e Train R2: %.4f \t Test R: %.4f \t Nonzero coef: %d" % (a, lasso.score(x_train, trpgen_train), 
                                                                                  lasso.score(x_test, trpgen_test), 
                                                                                  np.sum(lasso.coef_ != 0)))
#     with open(out_dir+"BA_"+variable_names[-1]+".csv", "a") as f:
#         f.write("%.6f,%.4f,%.4f,%s,%d,%d\n" % (a, 
#             lasso.score(x_train, trpgen_train), lasso.score(x_test, trpgen_test), 'lasso', 
#             np.sum(lasso.coef_ != 0), len(lasso.coef_)))

Parameter: 0.00e+00 Train R2: 0.0706 	 Test R: -0.2020 	 Nonzero coef: 64
Parameter: 1.00e-04 Train R2: 0.0706 	 Test R: -0.2016 	 Nonzero coef: 64
Parameter: 6.00e-03 Train R2: 0.0702 	 Test R: -0.1798 	 Nonzero coef: 62
Parameter: 7.00e-03 Train R2: 0.0701 	 Test R: -0.1762 	 Nonzero coef: 62
Parameter: 8.00e-03 Train R2: 0.0699 	 Test R: -0.1729 	 Nonzero coef: 58
Parameter: 1.00e-02 Train R2: 0.0696 	 Test R: -0.1664 	 Nonzero coef: 56
Parameter: 1.10e-02 Train R2: 0.0694 	 Test R: -0.1632 	 Nonzero coef: 55
Parameter: 1.20e-02 Train R2: 0.0692 	 Test R: -0.1601 	 Nonzero coef: 55
Parameter: 1.30e-02 Train R2: 0.0690 	 Test R: -0.1571 	 Nonzero coef: 54
Parameter: 1.40e-02 Train R2: 0.0687 	 Test R: -0.1541 	 Nonzero coef: 53
Parameter: 1.50e-02 Train R2: 0.0685 	 Test R: -0.1512 	 Nonzero coef: 52
Parameter: 2.00e-02 Train R2: 0.0670 	 Test R: -0.1382 	 Nonzero coef: 50
Parameter: 5.00e-02 Train R2: 0.0547 	 Test R: -0.0708 	 Nonzero coef: 37


<ipython-input-33-aff3c5ecbb93>:3: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
  lasso.fit(x_train, trpgen_train)
/home/jtl/anaconda3/envs/qingyi/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:647: UserWarning: Coordinate descent with no regularization may lead to unexpected results and is discouraged.
  model = cd_fast.enet_coordinate_descent(
/home/jtl/anaconda3/envs/qingyi/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.687e+05, tolerance: 3.630e+01 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in sklearn.linear_model.Ridge/RidgeCV instead.
  model = cd_fast.enet_coordinate_descent

In [34]:
# Ridge

for a in (1e+1)*np.array([0,0.1,1,2,3,4,5,6,7,8,10,20,50]):

    ridge = linear_model.Ridge(alpha=a)
    ridge.fit(x_train, trpgen_train)
#     with open(out_dir+sampling+"_"+model_code+"_regression_"+variable_names[-1]+".csv", "a") as f:
#         f.write("%s,%s,%s,%.5f,%.4f,%.4f,%s,%s,%d,%d\n" % (model_run_date, model_type, variable_names[-1], a, 
#             ridge.score(x_train, trpgen_train), ridge.score(x_test, trpgen_test), 'ridge', zoomlevel,
#             np.sum(ridge.coef_ != 0), len(ridge.coef_)))
    print("Parameter: %.2e Train R2: %.4f \t Test R: %.4f" % (a, ridge.score(x_train, trpgen_train), 
                                                              ridge.score(x_test, trpgen_test)))

Parameter: 0.00e+00 Train R2: 0.0706 	 Test R: -0.2020
Parameter: 1.00e+00 Train R2: 0.0705 	 Test R: -0.1890
Parameter: 1.00e+01 Train R2: 0.0664 	 Test R: -0.1155
Parameter: 2.00e+01 Train R2: 0.0603 	 Test R: -0.0760
Parameter: 3.00e+01 Train R2: 0.0549 	 Test R: -0.0538
Parameter: 4.00e+01 Train R2: 0.0503 	 Test R: -0.0401
Parameter: 5.00e+01 Train R2: 0.0465 	 Test R: -0.0309
Parameter: 6.00e+01 Train R2: 0.0432 	 Test R: -0.0244
Parameter: 7.00e+01 Train R2: 0.0404 	 Test R: -0.0198
Parameter: 8.00e+01 Train R2: 0.0379 	 Test R: -0.0163
Parameter: 1.00e+02 Train R2: 0.0339 	 Test R: -0.0117
Parameter: 2.00e+02 Train R2: 0.0225 	 Test R: -0.0044
Parameter: 5.00e+02 Train R2: 0.0119 	 Test R: -0.0034


# 2. MNL for Mode Share

In [9]:
# dataloader and model definition

trainset = SurveyDataset(torch.tensor(x_train,  dtype=torch.float), torch.tensor(y_train, dtype=torch.float))
trainloader = DataLoader(trainset, batch_size=len(trainset), shuffle=False)

testset = SurveyDataset(torch.tensor(x_test, dtype=torch.float), torch.tensor(y_test, dtype=torch.float))
testloader = DataLoader(testset, batch_size=len(testset), shuffle=False)

kldivloss = nn.KLDivLoss(reduction='sum')
mseloss = nn.MSELoss(reduction='none')

In [10]:
sst_train = np.sum(np.power(y_train - np.mean(y_train, axis=0), 2), axis=0)
sst_test = np.sum(np.power(y_test - np.mean(y_test, axis=0), 2), axis=0)

In [11]:
def mnl_torch(lr_list, wd_list):
    
    for (lr, wd) in itertools.product(lr_list, wd_list):
        
        print(f"[lr: {lr:.4f}, wd: {wd:3.2e}]")

        # model setup
        model = mnl.MNL(n_alts=4, n_features=x_train.shape[-1])
        optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=wd)

#         print(optimizer)
        # model training

        converged = 0
        ref1 = 0
        ref2 = 0

        for epoch in range(5000):

            kl_ = 0
            mse_ = 0
            mse1_ = 0
            mse2_ = 0
            mse3_ = 0
            mse4_ = 0

            for batch, (x_batch, y_batch) in enumerate(trainloader):
                
                # Compute prediction and loss
                util = model(x_batch)
                probs = torch.log(nn.functional.softmax(util, dim=1))
                kl = kldivloss(probs, y_batch)
        #         kl = kldivloss(torch.log(util), y_batch)
                kl_ += kl.item()

                mse = mseloss(torch.exp(probs), y_batch)
        #         mse = mseloss(util, y_batch)
                mse_ += mse.sum().item()
                mse1_ += mse[:,0].sum().item()
                mse2_ += mse[:,1].sum().item()
                mse3_ += mse[:,2].sum().item()
                mse4_ += mse[:,3].sum().item()
                mse = mse.sum()

                # Backpropagation
                optimizer.zero_grad()
                kl.backward()
                optimizer.step()

            train_kl = kl_/len(trainset)
            train_mse = np.sqrt(mse_/len(trainset))
            train_mse1 = np.sqrt(mse1_/len(trainset))
            train_mse2 = np.sqrt(mse2_/len(trainset))
            train_mse3 = np.sqrt(mse3_/len(trainset))
            train_mse4 = np.sqrt(mse4_/len(trainset))

            train_r1 = 1-mse1_/sst_train[0]
            train_r2 = 1-mse2_/sst_train[1]
            train_r3 = 1-mse3_/sst_train[2]
            train_r4 = 1-mse4_/sst_train[3]

            loss_ = train_kl

            if epoch % 5 == 0:

                kl_ = 0
                mse_ = 0 
                mse1_ = 0
                mse2_ = 0
                mse3_ = 0
                mse4_ = 0

                for batch, (x_batch, y_batch) in enumerate(testloader):
                    
                    util = model(x_batch)
                    probs = torch.log(nn.functional.softmax(util,dim=1))
                    kl = kldivloss(probs, y_batch)
            #         kl = kldivloss(torch.log(util), y_batch)
                    kl_ += kl.item()

                    mse = mseloss(torch.exp(probs), y_batch)
            #         mse = mseloss(util, y_batch)
                    mse_ += mse.sum().item()
                    mse1_ += mse[:,0].sum().item()
                    mse2_ += mse[:,1].sum().item()
                    mse3_ += mse[:,2].sum().item()
                    mse4_ += mse[:,3].sum().item()

                test_kl = kl_/len(testset)
                test_mse = np.sqrt(mse_/len(testset))
                test_mse1 = np.sqrt(mse1_/len(testset))
                test_mse2 = np.sqrt(mse2_/len(testset))
                test_mse3 = np.sqrt(mse3_/len(testset))
                test_mse4 = np.sqrt(mse4_/len(testset))

                r1 = r2_score(y_batch.numpy()[:,0],torch.exp(probs).detach().numpy()[:,0])
                r2 = r2_score(y_batch.numpy()[:,1],torch.exp(probs).detach().numpy()[:,1])
                r3 = r2_score(y_batch.numpy()[:,2],torch.exp(probs).detach().numpy()[:,2])
                r4 = r2_score(y_batch.numpy()[:,3],torch.exp(probs).detach().numpy()[:,3])

                if epoch >= 40:
                    if (np.abs(loss_ - ref1)/ref1<0.001) & (np.abs(loss_ - ref2)/ref2<0.001):
                        converged = 1
                        print("Early stopping at epoch", epoch)
                        break
                    if (ref1 < loss_) & (ref1 < ref2):
                        print("Diverging. stop.")
                        break
                    if loss_ < best:
                        best = loss_
                        best_epoch = epoch
                        output = (best_epoch, train_kl, train_mse, train_mse1, train_mse2, train_mse3, train_mse4,
                                  test_kl, test_mse, test_mse1, test_mse2, test_mse3, test_mse4,
                                  train_r1, train_r2, train_r3, train_r4, r1, r2, r3, r4)
                else:
                    best = loss_
                    best_epoch = epoch
                    output = (best_epoch, train_kl, train_mse, train_mse1, train_mse2, train_mse3, train_mse4,
                                  test_kl, test_mse, test_mse1, test_mse2, test_mse3, test_mse4,
                                  train_r1, train_r2, train_r3, train_r4, r1, r2, r3, r4)
                ref2 = ref1
                ref1 = loss_

            if epoch % 300 == 0:

                print(f"[epoch: {epoch:>3d}] Train KL loss: {train_kl:.3f} RMSE {train_mse:.3f}")
                   # {train_mse1:.3f} {train_mse2:.3f} {train_mse3:.3f} {train_mse4:.3f}")
                print(f"\t\t\t\t\t\t Train R2 score: {train_r1:.3f} {train_r2:.3f} {train_r3:.3f} {train_r4:.3f} ")
                print(f"[epoch: {epoch:>3d}] Test KL loss: {kl_/len(testset):.3f} RMSE {np.sqrt(mse_/len(testset)):.3f}")
                   #     {np.sqrt(mse1_/len(testset)):.3f} {np.sqrt(mse2_/len(testset)):.3f} {np.sqrt(mse3_/len(testset)):.3f} {np.sqrt(mse4_/len(testset)):.3f}")
                print(f"\t\t\t\t\t\t Test R2 score: {r1:.3f} {r2:.3f} {r3:.3f} {r4:.3f} ")

                print(f"[epoch: {epoch:>3d}] Train KL loss: {train_kl:.3f} Train R2 score: {train_r1:.3f} {train_r2:.3f} {train_r3:.3f} {train_r4:.3f} ")
                print(f"[epoch: {epoch:>3d}] Test KL loss: {kl_/len(testset):.3f} Test R2 score: {r1:.3f} {r2:.3f} {r3:.3f} {r4:.3f} ")

        with open(out_dir+"SAE_A_MNL.csv", "a") as f:
            f.write("%.1E,%.1E,%.1E,%d,%.4f,%.4f,%.4f,%.4f,%.4f,%.4f,%.4f,%.4f,%.4f,%.4f,%.4f,%.4f,%.4f,%.4f,%.4f,%.4f,%.4f,%.4f,%.4f,%.4f,%d\n" % 
                    ((weight,lr,wd)+output+(converged,)))

        print(f"[epoch: {best_epoch:>3d}] Train KL loss: {output[1]:.3f} Train R2 score: {output[13]:.3f} {output[14]:.3f} {output[15]:.3f} {output[16]:.3f} ")
        print(f"[epoch: {best_epoch:>3d}] Test KL loss: {output[7]:.3f} Test R2 score: {output[17]:.3f} {output[18]:.3f} {output[19]:.3f} {output[20]:.3f} ")
        print()
        
    return model

In [15]:
for i in range(1):
#     mnl_torch(lr_list=[1e-4], wd_list=[1e-3]);
#     mnl_torch(lr_list=[1e-4], wd_list=[1e-2]);

    model = mnl_torch(lr_list=[1e-4], wd_list=[1e-1]);
#     model = mnl_torch(lr_list=[1e-5], wd_list=[1e+0]);
#     mnl_torch(lr_list=[1e-4], wd_list=[0.1,1,10,50,100,1000]);
#     model = mnl_torch(lr_list=[1e-5], wd_list=[1e+1]);
#     mnl_torch(lr_list=[1e-4], wd_list=[50]);

#     mnl_torch(lr_list=[1e-4], wd_list=[1e+2]);

#     mnl_torch(lr_list=[5e-5], wd_list=[1e+3]);

[lr: 0.0001, wd: 1.00e-01]
[epoch:   0] Train KL loss: 0.782 RMSE 0.657
						 Train R2 score: -1.691 -4.592 -6.467 -1.708 
[epoch:   0] Test KL loss: 0.279 RMSE 0.346
						 Test R2 score: -0.140 -0.483 -2.953 -0.106 
[epoch:   0] Train KL loss: 0.782 Train R2 score: -1.691 -4.592 -6.467 -1.708 
[epoch:   0] Test KL loss: 0.279 Test R2 score: -0.140 -0.483 -2.953 -0.106 
[epoch: 300] Train KL loss: 0.150 RMSE 0.225
						 Train R2 score: 0.420 0.528 0.004 0.437 
[epoch: 300] Test KL loss: 0.125 RMSE 0.203
						 Test R2 score: 0.479 0.578 -0.073 0.412 
[epoch: 300] Train KL loss: 0.150 Train R2 score: 0.420 0.528 0.004 0.437 
[epoch: 300] Test KL loss: 0.125 Test R2 score: 0.479 0.578 -0.073 0.412 
[epoch: 600] Train KL loss: 0.141 RMSE 0.214
						 Train R2 score: 0.466 0.582 0.010 0.487 
[epoch: 600] Test KL loss: 0.116 RMSE 0.192
						 Test R2 score: 0.532 0.633 -0.103 0.448 
[epoch: 600] Train KL loss: 0.141 Train R2 score: 0.466 0.582 0.010 0.487 
[epoch: 600] Test KL loss: 0.11

In [16]:
torch.save(model.state_dict(), out_dir+"sae_a_D_1_220829.pt")
